In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers

current_learning_iteration = 0

# Device Class
class Device:
    def __init__(self, device_id, ram, storage, cpu, bandwidth, battery, charging):
        self.device_id = device_id
        self.ram = ram
        self.storage = storage
        self.cpu = cpu
        self.bandwidth = bandwidth
        self.battery = battery
        self.charging = charging
        self.energy_consumption = ram + storage + cpu + bandwidth
        self.model = self.create_model()
        self.last_round_participated = 0
        self.data = None  # Placeholder for dataset partition
        
        self.test_data = None
        # TODO: add test data for each device
        # TODO: use the test data retrieved from load dataset from mnist and distribute it between devices and global model
        

        self.number_of_times_fitted = 0

    def create_model(self):
        model = keras.Sequential([
            layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])
        model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
                      loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model
    



# Global Model
# Define the global model with the same architecture
def create_global_model():
    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

global_model = create_global_model()



import numpy as np

def aggregate_weights(global_model, devices):
    """Computes the weighted average of model weights from all devices and updates the global model."""

    num_devices = len(devices)
    if num_devices == 0:
        print("No devices available for aggregation.")
        return

    # Get device weights and participation ratios
    device_weights = [device.model.get_weights() for device in devices]
    device_participation_ratio = np.array(
        [device.last_round_participated / current_learning_iteration for device in devices]
    )

    # Total weight for normalization
    total_weight = np.sum(device_participation_ratio)
    if total_weight == 0:
        print("Total weight is zero, cannot perform aggregation.")
        return

    for item in device_participation_ratio:
        print(item)
    
    
    len_total_devices_data = 0
    for device in devices:
        len_total_devices_data += len(device.data[0])

    # Compute weighted sum of weights
    weighted_sums = [
        np.sum(np.stack([device_weights[i][layer] * device_participation_ratio[i] * len(devices[i].data[0])/float(len_total_devices_data) for i in range(num_devices)]), axis=0)
        for layer in range(len(device_weights[0]))
    ]
    

    # Compute weighted average
    weighted_avg_weights = [layer_sum / total_weight for layer_sum in weighted_sums]

    # Set the global model's weights to the weighted averaged weights
    print("old global model weights:", global_model.get_weights())
    global_model.set_weights(weighted_avg_weights)
    print("new global model weights:", global_model.get_weights())
    print(np.array_equal(global_model.get_weights(), weighted_avg_weights))
    # print(global_model.get_weights())
    # for i in range(100):
    #     print()
    # print(weighted_avg_weights)

d:\Github Repos\FL\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Load dataset from CSV
csv_file = 'devices.csv'
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip().str.lower()


# Convert CSV rows into device objects
devices = []
for _, row in df.iterrows():
    device = Device(
        row['id'], row['ram'], row['storage'], row['cpu'], row['bandwidth'], row['battery'],
        row.get('charging', 0)
    )
    devices.append(device)


# Load MNIST dataset
(x_train, y_train), (_, _) = keras.datasets.mnist.load_data()

# Normalize data and reshape for CNN
x_train = x_train.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)  # Add channel dimension

# Shuffle data
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train, y_train = x_train[indices], y_train[indices]

# Split into global test set (20%) and training set (80%)
split_index = int(0.8 * len(x_train))
x_train_devices, y_train_devices = x_train[:split_index], y_train[:split_index]
x_test_global, y_test_global = x_train[split_index:], y_train[split_index:]


# Split dataset among devices
num_devices = len(devices)
split_size = len(x_train_devices) // num_devices

for i, device in enumerate(devices):
    start = i * split_size
    end = (i + 1) * split_size if i < num_devices - 1 else len(x_train_devices)
    device.data = (x_train_devices[start:end], y_train_devices[start:end])


with open('bitstring.txt', 'r') as f:
    bitstring = f.read()

bitstring = [int(bit) for bit in bitstring.split(',')]

current_learning_iteration += 1
for device in devices:
    print(int(device.device_id))
    if bitstring[int(device.device_id)] == 1:
        device.model.fit(device.data[0], device.data[1], epochs=2, verbose=1)
        device.number_of_times_fitted += 1
        device.last_round_participated = current_learning_iteration


# Call this function after training the local models:
aggregate_weights(global_model, devices)


test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

########################################################
# # Update device participation based on the bitstring
# selected_devices = [device for device in devices if bitstring[int(device.device_id)] == 1]

# current_learning_iteration += 1
# # Train local models for selected devices
# for device in selected_devices:
#     device.model.fit(device.data[0], device.data[1], epochs=1, verbose=0)
#     device.number_of_times_fitted += 1
#     device.last_round_participated = current_learning_iteration

# # Aggregate weights to update the global model
# aggregate_weights(self.global_model, selected_devices)

# # Distribute the updated global model back to all devices
# for device in devices:
#     device.model.set_weights(self.global_model.get_weights())

# test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
# print(f"Global Model Accuracy: {test_acc:.4f}")

#################################################################

d:\Github Repos\FL\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0
Epoch 1/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1126 - loss: 2.3011  
Epoch 2/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2560 - loss: 2.2228 
1
Epoch 1/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1510 - loss: 2.2923  
Epoch 2/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1869 - loss: 2.2345 
2
Epoch 1/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0741 - loss: 2.3036      
Epoch 2/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3135 - loss: 2.2351 
3
Epoch 1/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1233 - loss: 2.3006  
Epoch 2/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2343 - loss: 2.2517 
4
Epoch 1/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1362 - loss: 2.2844  
Epoch 2/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3204 - loss: 2.2028 
5
Epoch 1/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2107 - loss: 2.2899  
Epoch 2/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc

In [4]:
import numpy as np
import random
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.selection.tournament import TournamentSelection
from pymoo.termination.default import DefaultMultiObjectiveTermination

# Parameters
NUM_DEVICES = num_devices   # Number of devices (length of bitstring)
POPULATION_SIZE = 100
NUM_GENERATIONS = 10

# Step 1: Define the Problem
import numpy as np
from pymoo.core.problem import Problem

class FederatedLearningProblem(Problem):
    def __init__(self, num_devices, devices, global_model, x_test_global, y_test_global):
        super().__init__(
            n_var=num_devices,         # Number of variables (bitstring length)
            n_obj=3,                   # Number of objectives
            n_constr=0,                # No constraints
            xl=np.zeros(num_devices),  # Lower bound (0)
            xu=np.ones(num_devices),   # Upper bound (1)
            type_var=np.bool_          # Binary variables (bitstrings)
        )
        self.devices = devices
        self.global_model = global_model
        self.x_test_global = x_test_global
        self.y_test_global = y_test_global

        # Save the initial global model weights
        self.initial_global_weights = global_model.get_weights()

    def _evaluate(self, X, out, *args, **kwargs):
        """Evaluates objective values for each solution in the population."""
        num_solutions = len(X)
        F = np.zeros((num_solutions, 3))  # Initialize objective matrix

        for i, bitstring in enumerate(X):
            # Reset the global model to its initial state
            self.global_model.set_weights(self.initial_global_weights)
            # Update device participation based on the bitstring
            selected_devices = [device for device, bit in zip(self.devices, bitstring) if bit == 1]
            # Objective 1: Hardware Objectives (maximize)
            hardware_score = sum(
                device.ram + device.storage + device.cpu + device.bandwidth + device.battery + device.charging
                for device in selected_devices
            )
            F[i, 0] = ((6 - hardware_score)/6.0)  # Minimize (negative of hardware score)
            # Objective 2: Fairness (prioritize devices with lowest local accuracy)
            local_accuracies = []
            for device in self.devices:
              # just the devices in this solution:
                if bitstring[int(device.device_id)] == 1:
                  _, accuracy = global_model.evaluate(device.data[0], device.data[1], verbose=0)
                  local_accuracies.append(accuracy)
                else:
                  local_accuracies.append(0)
            # Fairness score: Sum of (1 - accuracy) for selected devices should be minimized
            # This prioritizes devices with lower local accuracy

            fairness_score = 0  # Initialize fairness score

            for j, bit in enumerate(bitstring):
                # print(j)
                # print(bit)
                if bit == 1:  # Check if the device is selected
                    fairness_score += local_accuracies[j]  # Add the corresponding accuracy value
            # Now, fairness_score contains the final sum

            # fairness_score = sum(local_accuracies[j] for j, bit in enumerate(bitstring) if bit == 1)
            F[i, 1] = fairness_score/float(len(selected_devices))  # Minimize (negative of fairness score)  # Added (/Selected Devices) to normalize between 0 and 1
            # Objective 3: Global Model Accuracy (maximize)
            _, global_accuracy = self.global_model.evaluate(self.x_test_global, self.y_test_global, verbose=0)
            F[i, 2] = 1 - global_accuracy  # Minimize (1 - accuracy)
        out["F"] = F  # Set the objective values

'''
            # Update device participation based on the bitstring
            selected_devices = [device for device, bit in zip(self.devices, bitstring) if bit == 1]

            # Train local models for selected devices
            for device in selected_devices:
                device.model.fit(device.data[0], device.data[1], epochs=1, verbose=0)

            # Aggregate weights to update the global model
            aggregate_weights(self.global_model, selected_devices)

            # Distribute the updated global model back to all devices
            for device in self.devices:
                device.model.set_weights(self.global_model.get_weights())
            '''

'\n            # Update device participation based on the bitstring\n            selected_devices = [device for device, bit in zip(self.devices, bitstring) if bit == 1]\n\n            # Train local models for selected devices\n            for device in selected_devices:\n                device.model.fit(device.data[0], device.data[1], epochs=1, verbose=0)\n\n            # Aggregate weights to update the global model\n            aggregate_weights(self.global_model, selected_devices)\n\n            # Distribute the updated global model back to all devices\n            for device in self.devices:\n                device.model.set_weights(self.global_model.get_weights())\n            '

In [5]:


problem = FederatedLearningProblem(
    num_devices=NUM_DEVICES,
    devices=devices,
    global_model=global_model,
    x_test_global=x_test_global,
    y_test_global=y_test_global
)


# Step 2: Configure NSGA-II Algorithm
algorithm = NSGA2(
    pop_size=POPULATION_SIZE,
    sampling=BinaryRandomSampling(),      # Random bitstrings
    crossover=TwoPointCrossover(),        # Two-point crossover
    mutation=BitflipMutation(),           # Bit flip mutation
    eliminate_duplicates=True             # Avoid duplicate solutions
)


NUMBER_OF_LEARNING_ITERATIONS = 2

In [6]:
# Step 3: Run Optimization
res = minimize(
    problem=problem,
    algorithm=algorithm,
    # termination=MultiObjectiveSpaceToleranceTermination(tol=1e-6, n_last=10, nth_gen=5, n_max_gen=NUM_GENERATIONS),
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      6 |             - |             -
     2 |      200 |      9 |  0.1672845678 |         nadir
     3 |      300 |      9 |  0.1356322545 |         ideal


KeyboardInterrupt: 

In [ ]:
# Step 4: Extract the Best Pareto Front
pareto_front = res.F   # Objective values of solutions in Pareto front
pareto_solutions = res.X  # Corresponding bitstrings

# Print the Best Pareto Front Solutions
print("Best Pareto Front (Bitstrings):")
for bitstring in pareto_solutions:
    print("".join(map(str, bitstring)).replace('True','1').replace('False','0'))

bitstring = pareto_solutions[0] # for now!
bitstring = str(bitstring).replace('False','0').replace('True','1')
for char in bitstring:
    if char != '0' and char != '1':
        bitstring = bitstring.replace(char,'')

print(len(bitstring))
print(bitstring)
temp_bitstring = []
for bit in bitstring:
    temp_bitstring.append(bit)
bitstring = temp_bitstring

########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# Aggregate weights to update the global model
# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.model.fit(device.data[0], device.data[1], epochs=5, verbose=0)
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration


test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

Best Pareto Front (Bitstrings):
0011001011001000101011011011011001001010011001000100000010111101101110000000000010111000010000001110
1101010011010000111100010111001111011100111100111000111111001111011100101000101110100011111010101111
0011001011001000101011010111001101001010011001000100000010111101111110000001000000111000010000001110
0011001011001000101011011011011001011100111100100100000010000011110100101110000010100011111010101110
0011001011001000101011011011011001001010011001000100000010111101101110000001000000100011111010101110
0011001011001000101011011011010001011100111100100110000010000011111100101000101101011011111010101110
0011001011001000101011010111001101001010011001000100000010111101111110000001000000111000111010100110
0011001011001000101011011011011011011100111100111000111111001111011100101000101110100011111010101110
0011001011001000101011011011011001011100111100110000111111001111011100101000101110100011111010101110
00110010110010001010110101110011110111001111001110001111110

In [ ]:
print(bitstring) # for now!
print(len(bitstring))
print(bitstring.count("1"))

['0', '0', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0']
100
41


In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       10 |      4 |             - |             -
     2 |       20 |      7 |  0.2307692319 |         ideal
     3 |       30 |     10 |  0.0359156015 |         ideal
     4 |       40 |     10 |  0.1612903352 |         ideal
     5 |       50 |     10 |  0.0201756792 |             f
     6 |       60 |     10 |  0.0331854369 |             f
     7 |       70 |     10 |  0.0406118674 |             f
     8 |       80 |     10 |  0.0963562733 |         ideal
     9 |       90 |     10 |  0.0199798763 |             f
    10 |      100 |     10 |  0.0048348089 |         ideal


In [ ]:
########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)
# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()
aggregate_weights(global_model, selected_devices)
w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

False
Device 2.0: Loss before: [2.3014464378356934, 0.12708333134651184], Loss after: [2.2971396446228027, 0.12708333134651184]
Device 3.0: Loss before: [2.3013997077941895, 0.10625000298023224], Loss after: [2.2967536449432373, 0.22708334028720856]
Device 6.0: Loss before: [2.3017570972442627, 0.09791667014360428], Loss after: [2.296715497970581, 0.14166666567325592]
Device 8.0: Loss before: [2.3015942573547363, 0.09583333134651184], Loss after: [2.296424150466919, 0.125]
Device 9.0: Loss before: [2.3014817237854004, 0.11249999701976776], Loss after: [2.2969343662261963, 0.11249999701976776]
Device 12.0: Loss before: [2.3015387058258057, 0.09583333134651184], Loss after: [2.2940421104431152, 0.34583333134651184]
Device 16.0: Loss before: [2.3022358417510986, 0.11041666567325592], Loss after: [2.295330762863159, 0.13750000298023224]
Device 18.0: Loss before: [2.3010990619659424, 0.12291666865348816], Loss after: [2.2960875034332275, 0.23541666567325592]
Device 20.0: Loss before: [2.301

In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)
# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()
aggregate_weights(global_model, selected_devices)
w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       10 |      7 |             - |             -
     2 |       20 |     10 |  0.1149033047 |         ideal
     3 |       30 |     10 |  0.1263791392 |         ideal
     4 |       40 |     10 |  0.0446626673 |             f
     5 |       50 |     10 |  0.0531813837 |         ideal
     6 |       60 |     10 |  0.0627976867 |         ideal
     7 |       70 |     10 |  0.0666441130 |         ideal
     8 |       80 |     10 |  0.0185034611 |             f
     9 |       90 |     10 |  0.0322993886 |             f
    10 |      100 |     10 |  0.0970137703 |         ideal
False
Device 2.0: Loss before: [2.297051191329956, 0.12708333134651184], Loss after: [2.2878005504608154, 0.12708333134651184]
Device 3.0: Loss before: [2.2966902256011963, 0.10625000298023224], Loss after: [2.28582763671875, 0.30416667461395264]
Device 6.0: Loss before: [2.2979109287261963, 0.09791667014360428], Loss after: [2.2868010997772217, 0.1

In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)
# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()
aggregate_weights(global_model, selected_devices)
w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       10 |      6 |             - |             -
     2 |       20 |     10 |  0.1140514774 |         ideal
     3 |       30 |     10 |  0.0191277690 |         ideal
     4 |       40 |     10 |  0.0872905024 |         ideal
     5 |       50 |     10 |  0.0777815630 |         ideal
     6 |       60 |     10 |  0.0156152389 |         ideal
     7 |       70 |     10 |  0.0436081211 |         ideal
     8 |       80 |     10 |  0.0334872969 |         ideal
     9 |       90 |     10 |  0.0040579713 |         nadir
    10 |      100 |     10 |  0.0163946562 |             f
False
Device 2.0: Loss before: [2.2921078205108643, 0.13124999403953552], Loss after: [2.2730791568756104, 0.20208333432674408]
Device 3.0: Loss before: [2.291351318359375, 0.12083332985639572], Loss after: [2.267686605453491, 0.44583332538604736]
Device 6.0: Loss before: [2.2930214405059814, 0.1041666641831398], Loss after: [2.2700908184051514, 0.1

In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)
# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()
aggregate_weights(global_model, selected_devices)
w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       10 |      6 |             - |             -
     2 |       20 |     10 |  0.0962540945 |         ideal
     3 |       30 |     10 |  0.1702600751 |         ideal
     4 |       40 |     10 |  0.0457933994 |         ideal
     5 |       50 |     10 |  0.0358850306 |             f
     6 |       60 |     10 |  0.0693756153 |         ideal
     7 |       70 |     10 |  0.0096587875 |             f
     8 |       80 |     10 |  0.0514744528 |         ideal
     9 |       90 |     10 |  0.0085521767 |             f
    10 |      100 |     10 |  0.0276725565 |             f
False
Device 2.0: Loss before: [2.279834747314453, 0.31458333134651184], Loss after: [2.2243316173553467, 0.5]
Device 3.0: Loss before: [2.277980327606201, 0.2874999940395355], Loss after: [2.204665184020996, 0.512499988079071]
Device 6.0: Loss before: [2.2804510593414307, 0.2916666567325592], Loss after: [2.213855504989624, 0.24583333730697632]
Dev

In [ ]:
# ########################################################
# # Update device participation based on the bitstring
# selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# # Aggregate weights to update the global model
# aggregate_weights(global_model, selected_devices)

# # Distribute the updated global model back to all devices
# for device in devices:
#     device.model.set_weights(global_model.get_weights())

# current_learning_iteration += 1
# # Train local models for selected devices
# for device in selected_devices:
#     device.model.fit(device.data[0], device.data[1], epochs=5, verbose=0)
#     device.number_of_times_fitted += 1
#     device.last_round_participated = current_learning_iteration

# # Aggregate weights to update the global model
# aggregate_weights(global_model, selected_devices)

# test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
# print(f"Global Model Accuracy: {test_acc:.4f}")

In [ ]:
# ########################################################
# # Update device participation based on the bitstring
# selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
# # Distribute the updated global model back to all devices
# for device in devices:
#     device.model.set_weights(global_model.get_weights())

# current_learning_iteration += 1
# # Train local models for selected devices
# for device in selected_devices:
#     device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
#     device.number_of_times_fitted += 1
#     device.last_round_participated = current_learning_iteration

# # Aggregate weights to update the global model
# w1 = global_model.get_weights()
# aggregate_weights(global_model, selected_devices)
# w2 = global_model.get_weights()
# print("global model stayed the same?")
# print(np.array_equal(w1,w2))
# test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
# print(f"Global Model Accuracy: {test_acc:.4f}")

In [ ]:
# print("Before aggregation:", w1[0].flatten()[:5])
# aggregate_weights(global_model, selected_devices)
# print("After aggregation:", global_model.get_weights()[0].flatten()[:5])

In [ ]:
# ########################################################
# # Update device participation based on the bitstring
# selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
# # Distribute the updated global model back to all devices
# for device in devices:
#     device.model.set_weights(global_model.get_weights())

# current_learning_iteration += 1
# # Train local models for selected devices
# for device in selected_devices:
#     device.number_of_times_fitted += 1
#     device.last_round_participated = current_learning_iteration
#     loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
#     device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
#     loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
#     print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# # Aggregate weights to update the global model
# w1 = global_model.get_weights()
# aggregate_weights(global_model, selected_devices)
# w2 = global_model.get_weights()
# print("global model stayed the same?")
# print(np.array_equal(w1,w2))
# test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
# print(f"Global Model Accuracy: {test_acc:.4f}")

In [ ]:
# for i in range(NUMBER_OF_LEARNING_ITERATIONS):
#     # Step 3: Run Optimization
#     res = minimize(
#         problem=problem,
#         algorithm=algorithm,
#         # termination=MultiObjectiveSpaceToleranceTermination(tol=1e-6, n_last=10, nth_gen=5, n_max_gen=NUM_GENERATIONS),
#         termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
#         seed=42,
#         verbose=True
#     )


#     # Step 4: Extract the Best Pareto Front
#     pareto_front = res.F   # Objective values of solutions in Pareto front
#     pareto_solutions = res.X  # Corresponding bitstrings

#     # Print the Best Pareto Front Solutions
#     print("Best Pareto Front (Bitstrings):")
#     for bitstring in pareto_solutions:
#         print("".join(map(str, bitstring)))

#     bitstring = pareto_solutions[0] # for now!
#     print(bitstring)
#     # Convert to bitstring (list of 1s and 0s)
#     bitstring = [1 if word == "True" else 0 for word in str(bitstring).replace("True", "1 ").replace("False", "0 ").split()]


#     ########################################################
#     # Update device participation based on the bitstring
#     selected_devices = [device for device in devices if bitstring[int(device.device_id)] == 1]

#     # Aggregate weights to update the global model
#     aggregate_weights(global_model, selected_devices)

#     # Distribute the updated global model back to all devices
#     for device in devices:
#         device.model.set_weights(global_model.get_weights())

#     current_learning_iteration += 1
#     # Train local models for selected devices
#     for device in selected_devices:
#         device.model.fit(device.data[0], device.data[1], epochs=1, verbose=0)
#         device.number_of_times_fitted += 1
#         device.last_round_participated = current_learning_iteration


#     test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
#     print(f"Global Model Accuracy: {test_acc:.4f}")

#     ################################################################